In [ ]:
install.packages('harmony')

In [ ]:
library(Seurat)
library(cowplot)
library(harmony)

In [ ]:
WT <- readRDS('WTp13.rds')
KO <- readRDS('Mutp13.rds')

In [ ]:
WT@meta.data$Study = 'GSE162033'
WT@meta.data$Author = 'Levebvre'
WT@meta.data$Technology = " 3' v3 10x"
WT@meta.data$DevTP = "p13"
WT@meta.data$Strain = "Unknown"
WT@meta.data$Tissue = "Tibial and femur epiphyses"

WT@meta.data$`RNA_snn_res.0.5` <- NULL
WT@meta.data$seurat_clusters <- NULL
WT@meta.data$`RNA_snn_res.0.6` <- NULL
WT@meta.data$`RNA_snn_res.1.2` <- NULL
WT@meta.data$`RNA_snn_res.2` <- NULL
WT@meta.data$`RNA_snn_res.0.15` <- NULL
WT@meta.data$sum <- NULL
WT@meta.data$total <- NULL

WT@meta.data$percent.mt <- NULL
WT@meta.data$Age <- NULL
WT@meta.data$Organism= 'Mus musculus'
WT@meta.data$`detected.1`<- NULL
WT@meta.data$`subsets_ERCC_detected` <- NULL
WT@meta.data$`subsets_ERCC_percent` <- NULL
WT@meta.data$`subsets_ERCC_sum` <- NULL
WT@meta.data$`subsets_Mito_detected` <- NULL
WT@meta.data$`subsets_Mito_percent` <- NULL
WT@meta.data$`subsets_Mito_sum` <- NULL
WT@meta.data$`sum.1` <- NULL
WT@meta.data$`total.1` <- NULL
WT@meta.data$detected <- NULL
WT@meta.data$ident <- NULL
WT@meta.data$Instrument <- NULL
WT@meta.data$orig.ident <- NULL
WT@meta.data$`RNA_snn_res.0.8`<- NULL
WT@meta.data$Sample_Name <- NULL
WT@meta.data$seurat_clusters <- NULL
WT@meta.data$subsets_ERCC_detected <- NULL
WT@meta.data$subsets_ERCC_percent <- NULL
WT@meta.data$subsets_ERCC_sum <- NULL
WT@meta.data$Strain= 'WT'

In [ ]:
WT@meta.data$Batch = 'GSM4930081'


In [ ]:
head(WT@meta.data)

In [ ]:
KO@meta.data$Study = 'GSE162033'
KO@meta.data$Batch = 'GSM4930083'
KO@meta.data$Author = 'Levebvre'
KO@meta.data$Technology = " 3' v3 10x"
KO@meta.data$DevTP = "p13"
KO@meta.data$Strain = "Unknown"
KO@meta.data$Tissue = "Tibial and femur epiphyses"

KO@meta.data$`RNA_snn_res.0.5` <- NULL
KO@meta.data$seurat_clusters <- NULL
KO@meta.data$`RNA_snn_res.0.6` <- NULL
KO@meta.data$`RNA_snn_res.1.2` <- NULL
KO@meta.data$`RNA_snn_res.2` <- NULL
KO@meta.data$`RNA_snn_res.0.15` <- NULL
KO@meta.data$sum <- NULL
KO@meta.data$total <- NULL

KO@meta.data$percent.mt <- NULL
KO@meta.data$Age <- NULL
KO@meta.data$Organism= 'Mus musculus'
KO@meta.data$`detected.1`<- NULL
KO@meta.data$`subsets_ERCC_detected` <- NULL
KO@meta.data$`subsets_ERCC_percent` <- NULL
KO@meta.data$`subsets_ERCC_sum` <- NULL
KO@meta.data$`subsets_Mito_detected` <- NULL
KO@meta.data$`subsets_Mito_percent` <- NULL
KO@meta.data$`subsets_Mito_sum` <- NULL
KO@meta.data$`sum.1` <- NULL
KO@meta.data$`total.1` <- NULL
KO@meta.data$detected <- NULL
KO@meta.data$ident <- NULL
KO@meta.data$Instrument <- NULL
KO@meta.data$orig.ident <- NULL
KO@meta.data$`RNA_snn_res.0.8`<- NULL
KO@meta.data$Sample_Name <- NULL
KO@meta.data$seurat_clusters <- NULL
KO@meta.data$subsets_ERCC_detected <- NULL
KO@meta.data$subsets_ERCC_percent <- NULL
KO@meta.data$subsets_ERCC_sum <- NULL
KO@meta.data$Strain = 'KO'


In [ ]:
head(KO@meta.data)

In [ ]:
merged <- merge(WT, KO)

In [ ]:
Idents(merged) <- merged$CellType

In [ ]:
merged <- subset(merged, idents=c('Chondroprogenitors','Resting zone chondrocytes','Proliferative chondrocytes','Pre-hypertrophic chondrocytes','Hypertrophic chondrocytes','Articular chondrocytes','Osteoprogenitors', 'Osteoblasts','Osteocytes','Periosteal Progenitors','Periosteal progenitors','Periosteum','Vascular endothelial cells'))

In [ ]:
merged <- NormalizeData(merged, normalization.method = "LogNormalize", scale.factor = 10000)
merged <- FindVariableFeatures(merged, selection.method = "vst", nfeatures = 2000)

In [ ]:
all.genes <- rownames(merged)
merged <- ScaleData(merged, features = all.genes)

In [ ]:
merged <- RunPCA(merged, features = VariableFeatures(object = merged), verbose = FALSE)


In [ ]:
options(repr.plot.height = 5, repr.plot.width = 12)
p1 <- DimPlot(object = merged, reduction = "pca", pt.size = .1, group.by = "Strain")
p2 <- VlnPlot(object = merged, features = "PC_1", group.by = "Strain",  pt.size = .1)
plot_grid(p1,p2)

In [ ]:
options(repr.plot.height = 2.5, repr.plot.width = 6)
merged <- merged %>% 
    RunHarmony("Strain", plot_convergence = TRUE)

In [ ]:
harmony_embeddings <- Embeddings(merged, 'harmony')
harmony_embeddings[1:5, 1:5]

In [ ]:
options(repr.plot.height = 5, repr.plot.width = 12)
p1 <- DimPlot(object = merged, reduction = "harmony", pt.size = .1, group.by = "Strain")
p2 <- VlnPlot(object = merged, features = "harmony_2", group.by = "Strain",pt.size = .1)
plot_grid(p1,p2)

In [ ]:
merged <- merged %>% 
    RunUMAP(reduction = "harmony", dims = 1:30) %>% 
    FindNeighbors(reduction = "harmony", dims = 1:30) %>% 
    FindClusters(resolution = 0.5) %>% 
    identity()

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 10)
DimPlot(merged, reduction = "umap", group.by = "Strain", pt.size = 1, split.by = 'Strain')

In [ ]:
Idents(merged) <- merged$CellType 

In [ ]:
merged <- RenameIdents(object = merged, `Chondroprogenitors` = "7 - Chondroprogenitors")

merged <- RenameIdents(object = merged, `Resting zone chondrocytes` = "9 - Resting zone chondrocytes")

merged <- RenameIdents(object = merged, `Proliferative chondrocytes` = "10 - Proliferative chondrocytes")
merged <- RenameIdents(object = merged, `Pre-hypertrophic chondrocytes` = "11 - Pre-hypertrophic chondrocytes")
merged <- RenameIdents(object = merged, `Hypertrophic chondrocytes` = "12 - Hypertrophic chondrocytes")
merged <- RenameIdents(object = merged, `Articular chondrocytes` = "13 - Articular chondrocytes")

#merged <- RenameIdents(object = merged, `Tenocytes` = "16 - Tenocytes")
merged <- RenameIdents(object = merged, `Osteoprogenitors` = "17 - Osteoprogenitors")
merged <- RenameIdents(object = merged, `Osteoblasts` = "19 - Osteoblasts")
merged <- RenameIdents(object = merged, `Osteocytes` = "21 - Osteocytes")
merged <- RenameIdents(object = merged, `Periosteal Progenitors` = "22 - Periosteal progenitors")
merged <- RenameIdents(object = merged, `Periosteal progenitors` = "22 - Periosteal progenitors")
merged <- RenameIdents(object = merged, `Periosteum` = "23 - Periosteum")
#merged <- RenameIdents(object = merged, `Perivascular cells` = "30 - Perivascular cells")

merged <- RenameIdents(object = merged, `Vascular endothelial cells` = "31 - Vascular endothelial cells")

#merged <- RenameIdents(object = merged, `Immune cells` = "35 - Immune cells")
#merged <- RenameIdents(object = merged, `Erythroid cells` = "36 - Erythroid cells")



In [ ]:
merged$NandC <- Idents(merged)

In [ ]:
Idents(merged) <- merged$CellType

In [ ]:
merged <- RenameIdents(object = merged, `Chondroprogenitors` = "7")

merged <- RenameIdents(object = merged, `Resting zone chondrocytes` = "9")

merged <- RenameIdents(object = merged, `Proliferative chondrocytes` = "10")
merged <- RenameIdents(object = merged, `Pre-hypertrophic chondrocytes` = "11")
merged <- RenameIdents(object = merged, `Hypertrophic chondrocytes` = "12")
merged <- RenameIdents(object = merged, `Articular chondrocytes` = "13")

#merged <- RenameIdents(object = merged, `Tenocytes` = "16")
merged <- RenameIdents(object = merged, `Osteoprogenitors` = "17")
merged <- RenameIdents(object = merged, `Osteoblasts` = "19")
merged <- RenameIdents(object = merged, `Osteocytes` = "21")
merged <- RenameIdents(object = merged, `Periosteal Progenitors` = "22")
merged <- RenameIdents(object = merged, `Periosteal progenitors` = "22")
merged <- RenameIdents(object = merged, `Periosteum` = "23")
#merged <- RenameIdents(object = merged, `Perivascular cells` = "30")

merged <- RenameIdents(object = merged, `Vascular endothelial cells` = "31")

#merged <- RenameIdents(object = merged, `Immune cells` = "35")
#merged <- RenameIdents(object = merged, `Erythroid cells` = "36")



In [ ]:
merged$Numbers <- Idents(merged)

In [ ]:
Idents(merged) <- merged$CellType

In [ ]:
suppressPackageStartupMessages({
    library(DropletUtils)
    library(SingleCellExperiment)
    library(scuttle)
    library(Seurat)
    library(SeuratWrappers)
    library(stringr)
    library(dplyr)
    library(data.table)
    library(Matrix)
    library(patchwork)
    library(ggplot2)
    library(sceasy)
})

options(repr.plot.width = 16, repr.plot.height = 8)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 10)
p1 <- DimPlot(merged, group.by='Numbers', label.color = 'white', label.box = TRUE, reduction = "umap",order = c('39','38','37','36','35','34','33','32','31','30','29','28','27','26','25','24','23','22','21','20','19','18','17','16','15','14','13','12','11','10','9','8','7','6','5','4','3','2','1'), pt.size = 0.3, label = T , label.size=8, repel=TRUE, raster=FALSE, 
              cols= c('1'='#66C2A5',
    '2'='#F781BF',
    '3'='#B3DE69',
    '4'='#8DA0CB',
    '5'='#949086',
    '6'='#FFCCCB',
    '7'='#666666',
    '8'='#B3B3B3',
    '9'='#8DD3C7',
    '10'='#D4A6CD',
    '11'='#FB8072',
    '12'='#FFD700',
    '13'='#0B0F5E',
    '14'='#377EB8',
    '15'='#FDB462',
    '16'='#DD2400',
    '17'='#FFFF33',
    '18'='#A65628',
    '19'='#FF7F50',
    '20'='#90EE90',
    '21'='#003e00',
    '22'='#FFD92F',
    '23'='#FF7F00',
    '24'='#502c49',
    '25'='#A6D854',
    '26'='#B3B3B3',
    '27'='#5E885e',
    '28'='#961900',
    '29'='#F845a3',
    '30'='#8DA0CB',
    '31'='#7e18c6',
    '32'='#B0D30A',
    '33'='#640738',
    '34'='#A2CFFE',
    '35'='#66C2A5',
    '36'='#4B0082',
    '37'='#B15928',
    '38'='#A0d4a4',
    '39'='#8B4513',
   '40'='#1F78B4',
    '41'='#4B0082',
    '42'='#AD8530')) +
        xlab("UMAP 1") + ylab("UMAP 2") +NoLegend() +
        theme(axis.title = element_text(size = 15)) +
        theme(plot.title = element_blank()) +
        theme(axis.text.x = element_blank(),
              axis.ticks.x = element_blank(),
              axis.text.y = element_blank(),
              axis.ticks.y = element_blank())    

p1 

In [ ]:
ggsave(file="/home/jovyan/researcher_home/Documents/Tom/Atlas/output/Subset_KOandWT_Numbers_Harmony_labels.svg", width=10, height=10)


In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
p1 <- DimPlot(merged, group.by='NandC',reduction = "umap",order = c('39 - Keratinocytes', '38 - Neurons', '37 - Platelets', '36 - Erythroid cells', '35 - Immune cells', '34 - Epithelial cells', '33 - AER', '32 - Lymphatic endothelial cells', '31 - Vascular endothelial cells', '30 - Perivascular cells', '29 - Skeletal muscle cells', '28 - Muscle progenitors', '27 - Myogenic stem cells', '26 - Fibroblasts', '25 - Osteoclasts', '24 - Endosteum', '23 - Periosteum', '22 - Periosteal progenitors', '21 - Osteocytes', '20 - Fast proliferating osteoblasts', '19 - Osteoblasts', '18 - Pre-osteoblasts', '17 - Osteoprogenitors', '16 - Tenocytes', '15 - Tenocyte precursors', '14 - Joint precursors', '13 - Articular chondrocytes', '12 - Hypertrophic chondrocytes', '11 - Pre-hypertrophic chondrocytes', '10 - Proliferative chondrocytes', '9 - Resting zone chondrocytes', '8 - Chondrocytes', '7 - Chondroprogenitors', '6 - Fast proliferating cells', '5 - ZPA', '4 - Distal limb bud mesenchyme', '3 - Intermediate limb bud mesenchyme', '2 - Proximal limb bud mesenchyme', '1 - BMSCs')
, pt.size = 0.3, split.by= 'Strain', label = F , label.size=6 ,label.color = 'black', repel=TRUE, raster=FALSE, cols= c(`1 - BMSCs` = "#66C2A5",
`2 - Proximal limb bud mesenchyme` = "#F781BF",
`3 - Intermediate limb bud mesenchyme` = "#B3DE69",
`4 - Distal limb bud mesenchyme` = "#8DA0CB",
`5 - ZPA` = "#949086",
`6 - Fast proliferating cells` = "#FFCCCB",
`7 - Chondroprogenitors` = "#666666",
`8 - Chondrocytes` = "#B3B3B3",
`9 - Resting zone chondrocytes` = "#8DD3C7",
`10 - Proliferative chondrocytes` = "#D4A6CD",
`11 - Pre-hypertrophic chondrocytes` = "#FB8072",
`12 - Hypertrophic chondrocytes` = "#FFD700",
`13 - Articular chondrocytes` = "#0B0F5E",
`14 - Joint precursors` = "#377EB8",
`15 - Tenocyte precursors` = "#FDB462",
`16 - Tenocytes` = "#DD2400",
`17 - Osteoprogenitors` = "#FFFF33",
`18 - Pre-osteoblasts` = "#A65628",
`19 - Osteoblasts` = "#FF7F50",
`20 - Fast proliferating osteoblasts` = "#90EE90",
`21 - Osteocytes` = "#003e00",
`22 - Periosteal progenitors` = "#FFD92F",
`23 - Periosteum` = "#FF7F00",
`24 - Endosteum` = "#502c49",
`25 - Osteoclasts` = "#A6D854",
`26 - Fibroblasts` = "#B3B3B3",
`27 - Myogenic stem cells` = "#5E885e",
`28 - Muscle progenitors` = "#961900",
`29 - Skeletal muscle cells` = "#F845a3",
`30 - Perivascular cells` = "#8DA0CB",
`31 - Vascular endothelial cells` = "#7e18c6",
`32 - Lymphatic endothelial cells` = "#B0D30A",
`33 - AER` = "#640738",
`34 - Epithelial cells` = "#A2CFFE",
`35 - Immune cells` = "#66C2A5",
`36 - Erythroid cells` = "#D4A6CD",
`37 - Platelets` = "#B15928",
`38 - Neurons` = "#A0d4a4",
`39 - Keratinocytes` = "#8B4513"
)) +
        xlab("UMAP 1") + ylab("UMAP 2") +
        theme(axis.title = element_text(size = 15)) +
        theme(plot.title = element_blank()) +
        theme(axis.text.x = element_blank(),
              axis.ticks.x = element_blank(),
              axis.text.y = element_blank(),
              axis.ticks.y = element_blank())    

p1 

In [ ]:
library(viridis)

In [ ]:
Idents(merged) <- merged$Strain

In [ ]:
merged$Strain <-as.factor(merged$Strain)

In [ ]:
head(merged$CellType)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
g2 <- WhichCells(merged, idents = ("WT"))
g3 <- WhichCells(merged, idents = ("KO"))

p1 <- DimPlot(merged,split.by='Strain', group.by='CellType',reduction = 'umap', label = FALSE , raster=F, label.size = 8, cells.highlight = list(g3,g2), cols.highlight = c("darkred", "darkred"),cols='grey') + xlab("UMAP 1") + ylab("UMAP 2") + NoLegend()
p1


